# Logistic Regression

### Complexity: O(n.n.m)

n = samples

m = features

In [ ]:
import pandas as pd

seed = 's1'

# Bipedal Hardcore
# base = '../../datalake/v0_1_3/standard/data/xbipedal/initialconditions'
# n_features = 200

# Double Pole
base = '../../../../datalake/data_baseline/data/xdpole/initialconditions'
n_features = 6

ic = pd.read_csv(f"{base}/{seed}_run.csv")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

features = ic.columns[[i for i in range(n_features)]]
target = 'performance'
X, y = ic[features], ic[target]

start = 1000 * 10
X, y = X[start:], y[start:]
X = PolynomialFeatures(2).fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
import numpy as np
from utils import fit_cv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

grid = {
    'C': np.logspace(-3,3,7),
    'penalty': ["l1","l2"],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
}
print(grid)

reg = GridSearchCV(
    LogisticRegression(),
    param_grid=grid,
    cv=3,
    verbose=0,
    n_jobs=-1
)

params = fit_cv(reg, (X_train, y_train), 'logistic')

In [ ]:
from critic import evolution

def model(X=[], y=[]):
    m = LogisticRegression(**params, random_state=42)
    if len(X) + len(y) > 0:
        m.fit(X,y)
    return m

s, p, t = evolution(X, y, model, batch=10)
print(t)

In [ ]:
import seaborn as sns
from utils import graph_defaults
graph_defaults()

sns.lineplot(s)

In [ ]:
p = [yp for gen in p for yp in gen]

sns.scatterplot(p)

In [ ]:
yt = [y[i] for i in range(start+1, len(y), 2)]

e = [p[i]/yt[i] for i in range(len(p))]

In [ ]:
sns.scatterplot(e)

In [ ]:
sns.histplot(e, bins=20)

In [ ]:
e_df = pd.DataFrame(e)
e_df.columns = ["e"]

e_clean = e_df.query("-1 < e <= 1")
e_outliers = e_df.query("1 < e or e <= -1")

print("outliers", len(e_outliers))
print("near", len(e_clean))

In [ ]:
e_outliers.hist()

In [ ]:
e_clean.hist()